# MMDb Functions

## Imports

In [6]:
## Basic
import pandas as pd
import numpy as np

import datetime as dt
from datetime import date
from collections import Counter
from itertools import combinations

import time
import requests
import json
import pickle

In [7]:
## Plotting
from IPython.core.display import HTML
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
## APIs
# https://imdbpy.readthedocs.io/en/latest/usage/quickstart.html
from imdb import IMDb

# https://pytrakt.readthedocs.io/en/latest/getstarted.html
from trakt import init
from trakt.users import User

import trakt.core

In [9]:
import warnings
warnings.filterwarnings('ignore')

## Classes

### Trakt

In [10]:
# https://trakt.docs.apiary.io/#
# Access Trakt
trakt.core.AUTH_METHOD = trakt.core.OAUTH_AUTH  # Set the auth method to OAuth
# Add trakt login parameters
user = ''
user_client_id = ''
user_client_secret = ''

In [11]:
# TRAKT class
class TRAKT:
    # Initialize dataframe and connect to trakt api
    def __init__(self):
        self.__df = pd.DataFrame()
        init(user, client_id=user_client_id, client_secret=user_client_secret);

    # Populate trakt dataframe with data since 'start_date'
    def create_df(self, start_date='2018-01-01'):
        # Access personal account
        self.__my = User(user)      
               
        temp = [
            [self.__my.watched_movies[i].title, 
             self.__my.watched_movies[i].imdb, 
             self.__my.watched_movies[i].last_watched_at, 
             self.__my.watched_movies[i].year] 
            for i in range(len(self.__my.watched_movies))
        ]
        
        timezone = 'Europe/Lisbon'
        
        # Add info to dataframe
        self.__df = self.__df.append(temp)
        self.__df.columns = ['title','id','timestamp','year']
                        
        self.__df.loc[:,'timestamp'] = pd.to_datetime(self.__df.timestamp, utc=True)
        # https://stackoverflow.com/questions/55598122/pandas-adding-timezone-offset-to-the-timestamp-after-using-tz-convert
        self.__df.loc[:,'timestamp'] = self.__df.loc[:,'timestamp'].dt.tz_convert(timezone).dt.tz_localize(None)
        self.__df = self.__df[self.__df.timestamp > pd.Timestamp(start_date)].sort_values(by='timestamp').reset_index(drop=True)

        return self.__df

### OMDb

In [12]:
# API key from Datacamp
# https://campus.datacamp.com/courses/intermediate-importing-data-in-python/interacting-with-apis-to-import-data-from-the-web-2?ex=7
apiKey = '72bc447a'
data_URL = 'http://www.omdbapi.com/?apikey='+apiKey

In [13]:
# OMDb class
class OMDB:
    # Initialize dataframe 
    def __init__(self):
        self.__df = pd.DataFrame()

    # Load OMDb file      
    def load(self, filename):
        return pd.read_csv(filename)
       
    # Convert 'N/A' data from int columns  
    def convert_to_int(self, x):
        # clean 'Year','Runtime','Metascore','imdbVotes'
        x = x.fillna(0)
        return x.replace('N/A', 0, regex=True).astype(int)

    ##
    # Correction and convertion of different columns 
    @staticmethod
    def convert_rated(x):
        return x.fillna('N/A').astype(str)
    
    @staticmethod
    def convert_imdb_rating(x):
        return x.apply(pd.to_numeric, errors='coerce').astype(float)
    
    def convert_imdb_votes(self, x):
        # remove ',' from votes (ex: 6,000 to 6000)
        return self.convert_to_int(x.astype(str).str.replace(',',''))
    
    @staticmethod
    def convert_imdb_id(x):
        # only numeric ID to join
        return x.astype(str).str.replace('tt','')
    
    def convert_runtime(self, x):
        # remove 'min', get only integers
        return self.convert_to_int(x.astype(str).replace('N/A', '000 min', regex=True).str.replace(r'\D', ''))
    
    @staticmethod
    def convert_timestamp(x):
        # remove timezone
        return pd.to_datetime(x) #.apply(lambda x: x.replace(tzinfo=None))

    def convert_boxoffice(self, x):
        # clean box office values
        return self.convert_to_int(x.astype(str).str.lstrip('$').str.replace(',',''))
    
    @staticmethod
    def convert_country(x):
        # convert country names
        replace_country = {
            'United Kingdom':'UK',
            'United States':'USA'
        }
        return x.replace(replace_country, regex=True)

    @staticmethod
    def convert_language(x):
        # convert language names
        replace_lang = {
            'American Sign Language':'American Sign',
            'American Sign':'American Sign Language',
            'Korean Sign':'Korean Sign Language'
        }
        return x.replace(replace_lang, regex=True)
    
    # Convert columns to appropriate formats
    def convert_columns(self, df):
        df['rated'] = self.convert_rated(df['rated'])
        df['runtime'] = self.convert_runtime(df['runtime'])
        df['language'] = self.convert_language(df['language'])
        df['country'] = self.convert_country(df['country'])
        df['metascore'] = self.convert_to_int(df['metascore'])
        df['imdbrating'] = self.convert_imdb_rating(df['imdbrating'])
        df['imdbvotes'] = self.convert_imdb_votes(df['imdbvotes'])
        df['imdbid'] = self.convert_imdb_id(df['imdbid'])
        df['timestamp'] = self.convert_timestamp(df['timestamp'] )   
        
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
            
        return df
            
    # Connect with OMDb API
    def request_db(self, df):        
        response = []
        movies = []
        
        for i in range(len(df)):
            params = {}
            movieID = df.iloc[i]['id']

            params = {
                'type':'movie',
                'i':movieID
            }
            
            movies.append(requests.get(data_URL, params=params).json())
        
        return movies

In [14]:
# Test queries
testing = False

if testing:
    # single movie request
    stitle = "Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb"
    idd = 'tt3581652'
    
    params = {
        #'t':stitle,
        'type':'movie',
        #'y': 2021 #syear
        'i':idd
    }
    
    responseTest = requests.get(data_URL, params=params).json()
    print(responseTest)

else:
    pass

### IMDb

In [15]:
# Extract these columns
# don't change this in order to get correct info from imdb api
cols_imdb = ['title','year','imdbID','runtimes','genres','director','writer','cinematographer','cast','countries','languages','rating','votes','plot outline','production companies']

In [16]:
# IMDb class
class IMDB:
    # Initialize dataframe 
    def __init__(self):
        self.__df = pd.DataFrame(columns=cols_imdb)
        
        self.__cols_int = ['year','runtimes','votes']
        self.__cols_str = ['imdbid','genres','director','writer','cinematographer','cast','countries','languages','production companies']
    
    # Load IMDb file      
    def load(self, filename):
        return pd.read_csv(filename)
       
    # Convert columns to appropriate formats
    def convert_columns(self, df):
        df[self.__cols_int] = df[self.__cols_int].replace('N/A', 0).astype(int) #.fillna(0).astype(int)
        df[self.__cols_str] = df[self.__cols_str].fillna('N/A').astype(str)
        df['date'] = pd.to_datetime(df['date'])
        
        return df
        
    # Extract information from imdb ia object : <info>
    @staticmethod    
    def get_info_list(x, cols, d):        
        # x - series
        # cols - columns
        # d - dictionary
        for i in range(len(cols)):
            if x[cols[i]] != None:
                d[cols[i]] = x[cols[i]]
            else:
                if (cols[i] == 'rating') or (cols[i] == 'votes'):
                    d[cols[i]] = 0
                elif cols[i] == 'plot outline':
                    d[cols[i]] = 'N/A'
                else:
                    d[cols[i]] = ['N/A']            
        return d

    # Extract information from imdb ia object : <name>
    @staticmethod
    def get_name_list(x, cols, d):
        # x - series
        # cols - columns
        # d - dictionary
        for i in range(len(cols)):
            if x[cols[i]] != None:
                temp = []
                for info in x[cols[i]]:
                    if info != None:
                        temp.append(info)
                    else:
                        pass
                # sometimes same person appears multiple times
                if cols[i] == 'writer':
                    # clean list
                    d[cols[i]] = list(filter(None, list(set(temp))))
                else:
                    d[cols[i]] = temp         
            else:
                d[cols[i]] = ['N/A']            
        return d

    # Clean output from db request    
    def clean_df(self, movies):  
        df = pd.DataFrame(columns=cols_imdb)
        
        # columns
        cols_info = ['runtimes','genres','countries','languages','rating','votes','plot outline']
        cols_name = ['director','writer','cinematographer','cast','production companies']
    
        # dictionaries
        dict_info = {}
        dict_name = {}
        
        for i in range(len(movies)):
            # intiliaze
            temp_movie = []
            movie = movies[i]

            # set values
            title = movie['title']
            year = movie['year']
            imdbID = movie['imdbID']
            
            dict_info = self.get_info_list(movie, cols_info, dict_info)
            dict_name = self.get_name_list(movie, cols_name, dict_name)
    
            # organize columns according to DF
            temp_movie = [title, year, imdbID, 
                          dict_info['runtimes'], dict_info['genres'], 
                          dict_name['director'], dict_name['writer'],
                          dict_name['cinematographer'], dict_name['cast'],
                          dict_info['countries'], dict_info['languages'], 
                          dict_info['rating'], dict_info['votes'], 
                          dict_info['plot outline'], dict_name['production companies']
                         ]
        
            df.loc[len(df)] = temp_movie
       
        df = list_to_string(df, cols_info[:-3])
        df = list_to_string(df, cols_name)    
        
        return df
    
    # Connect with IMDb API
    def request_db(self, movies): #, ids=False):
        all_movies = []  
        
        # Create an instance of the IMDb class to access API
        ia = IMDb()        

        for i in range(len(movies)):
            search = []
            
            temp = ia.get_movie(movies.iloc[i]['id'].replace(r'tt', ''))            
            # gather data to export
            all_movies.append({key: temp.get(key) for key in cols_imdb})
            
        return all_movies  

## General

In [17]:
# Dataframe styling
# https://www.analyticsvidhya.com/blog/2021/06/style-your-pandas-dataframe-and-make-it-stunning/

In [18]:
# Merge the two main dataframes
def merge_dfs(df_omdb, df_imdb):
    ## Prepare key to merge (remove tt from id and always get 7 numbers)
    # OMDb format imdb id 
    df_omdb['imdbid'] = df_omdb['imdbid'].str.replace(r'tt', '')
    df_omdb['imdbid'] = df_omdb['imdbid'].str.zfill(7)
    # IMDb format imdb id 
    df_imdb['imdbid'] = df_imdb['imdbid'].str.replace(r'tt', '')
    df_imdb['imdbid'] = df_imdb['imdbid'].str.zfill(7)
    
    # Merge dataframes on key = 'imdbid'
    df_merge = df_imdb.merge(df_omdb, on='imdbid')
    
    # Select columns after merge
    merge_cols = ['title_x','year_x','imdbid','rated','runtimes','genres','director_x','writer_x','cinematographer',
                  'cast','plot','languages','countries','metascore','rating','votes','production companies',
                  'timestamp_y','date_y','time_y']   
    
    df_merge = df_merge[merge_cols]

    # Rename columns
    cols = ['Title','Year','imdbID','Rated','Runtime','Genre','Director','Writer','Cinematographer',
            'Actors','Plot','Language','Country','Metascore','imdbRating','imdbVotes','Production',
            'Timestamp','Date','Time']   
    
    df_merge.columns = cols
    
    # Drop duplicates
    df_merge.drop_duplicates(['Title','Year','Date'], keep='first', inplace=True)
    # Sort by timestamp
    df_merge.sort_values(by='Timestamp', inplace=True)
    df_merge.reset_index(drop=True, inplace=True)
    
    return df_merge

In [19]:
# Example of preparation for "exploding"
# Convert string with multiple values to array of values
# df_explode_genre.Genre = df_explode_genre.Genre.str.split(',').apply(lambda x: [e.strip() for e in x])

# https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    
    return res

In [20]:
# Explode date into multiple columns
def explode_date(df):
    df['DateYear'] = df['Date'].dt.year
    
    df['Month'] = df['Date'].dt.month
    df['MonthName'] = df['Date'].dt.month_name()
    
    df['Weekday'] = df['Date'].dt.weekday
    df['WeekdayName'] = df['Date'].dt.strftime("%A")
    
    df['Day'] = df['Date'].dt.day
    #df.drop(columns='Date', inplace=True)
    
    return df

In [21]:
def split_explode(df, col):
    df[col] = df[col].str.split(',').apply(lambda x: [e.strip() for e in x])
    df = explode(df, [col])
    
    return df

In [22]:
# Convert column of lists into column of strings
def list_to_string(df, cols):
    for i in range(len(cols)):
        df[cols[i]] = df[cols[i]].agg(lambda x: ', '.join(map(str, x)))
        
    return df

In [23]:
# Get total movies watched by Year
def totals_by_year(df, all_watched=False, docs=False):
    df = df[['Title','Genre','Director','Date']].copy()
    # Get documentaries
    df_genres = df.Genre.str.contains('Documentary')
    
    if all_watched:
        df_ = df.copy()
    else:
        if docs:
            df_ = df[df_genres] # include documentaries
        else:
            df_ = df[~df_genres] # do not include documentaries

    # Get year
    df_['Year'] = df_['Date'].dt.year
    df_.drop(columns='Date', inplace=True)
    # To avoid incorrect match from movies with same title (watched in the same year, like Swan Song (2021))
    # Add 'Director' to 'Title' to create unique string
    df_['Title'] = df_['Title'] + ' - ' + df_['Director']
    df_.drop(columns='Director', inplace=True)
    
    # Group by Year
    df_by_year = df_.groupby(by='Year').count()
    # Unique and count all movies watched
    df_by_year_uc = df_.groupby(by='Year').agg({'Title':['nunique','count']})
    
    # Add total row
    rowtotal = df_by_year_uc.sum()
    rowtotal.name = 'All'
    df_by_year_uc.append(rowtotal)
    
    return df_by_year, df_by_year_uc

In [24]:
# Check if foo size is different than df_track size (new movies viewed)
def check_new_movies(foo, df_trakt):
    data = []
    newMovies = 0
    cols = ['title','year']
        
    if isinstance(foo, pd.core.frame.DataFrame) and get_data:
        # df columns as lowercase in order to compare
        foo.columns = [x.lower() for x in foo.columns]
        # last date in the dataset
        last_date = foo['date'].dt.date.max()
        # new movies since dataset's last date
        newMovies = len(df_trakt[df_trakt['timestamp'].dt.date > last_date])
    
        # get added movies
        if newMovies > 0:   
            diffMovies = list(set(df_trakt['id']) - set(foo['imdbid']))
            df_diffMovies = df_trakt[df_trakt['id'].str.contains('|'.join(diffMovies))]
            
            ## Select data to search in OMDB
            data = df_diffMovies[cols]
                        
    elif not isinstance(foo, pd.core.frame.DataFrame):       
        newMovies = len(df_trakt)
        ## Select data to search in OMDB
        data = df_trakt[cols]
        
    else:
        print('Do nothing.')
        
    return data, newMovies

In [25]:
# Extract date and time from timestamp
def from_timestamp(df):
    df['date'] = df['timestamp'].dt.date
    df['date'] = pd.to_datetime(df['date'])
    # extracting time from timestamp
    df['time'] = [dt.datetime.time(d) for d in df['timestamp']]    
    
    return df

In [26]:
# Check new data with existing df from pickle file
def look_iama_pickle(foo, df_trakt, api, forceRequest=False):
    movies = []
    
    # Check difference
    _, newMovies = check_new_movies(foo, df_trakt)   
    
    data = df_trakt.iloc[-newMovies:,:] #[['title','id']] # year
    
    if api == 'omdb':
        movies = omdb.request_db(data)
    elif api == 'imdb':
        movies = imdb.request_db(data)
        movies = imdb.clean_df(movies)
        
    index = data.index     
        
    # copy foo if pickle exists and there's no new movies
    if isinstance(foo, pd.core.frame.DataFrame) and not forceRequest:  
        if newMovies == 0:
            df = foo.copy()
            df['timestamp'] = pd.Series(df_trakt['timestamp'])
            
        else: # newMovies != 0
            df_movies = pd.DataFrame(movies)
            df_movies.columns = [x.lower() for x in df_movies.columns]

            df_movies = df_movies.tail(newMovies).reset_index(drop=True)
            
            index = index[-newMovies:]
            df_movies['timestamp'] = pd.Series(df_trakt.iloc[index].reset_index()['timestamp']) 
            df_movies = from_timestamp(df_movies)     
            
            df = foo.append(df_movies)
    
    elif not isinstance(foo, pd.core.frame.DataFrame) or forceRequest:
        df = pd.DataFrame(movies)
        # use OMDB query result and convert to dataframe
        df['timestamp'] = pd.Series(df_trakt['timestamp'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
        
        df = from_timestamp(df)
    
    else:
        pass

    return df.reset_index(drop=True)

In [27]:
# Single movie hard correct
def hard_correct_movie_data(df, loc, idt, api):
    temp = []
    response = []
    
    if api == 'omdb':
        if 'Error' in df.columns:
            df.drop('Error', axis=1, inplace=True)
        
        response = requests.get(data_URL, params={'type':'movie', 'i':idt}).json()
        temp.append(response)
        
        sel_cols = all_cols[:-4] # up until 'Response' - no 'Timestamp'
        correct = pd.DataFrame(temp)[sel_cols]
        df.loc[loc, sel_cols] = correct.loc[0] # only
    
    elif api == 'imdb':
        idt = idt.replace(r'tt', '')
        ia = IMDb()
        response = ia.get_movie(idt, info='main')
        temp.append({key: response.get(key) for key in cols_imdb})

        correct = imdb.clean_df(temp)
        df.iloc[loc, :] = correct.loc[0]
        
    else:
        pass
    
    return df

In [28]:
# Get days per month/per year without a movie watched
def skip_days(df, calendar, year):
    # explode df
    df_explode = explode_date(df)
    # filter by year
    df_explode = df_explode[df_explode['Date'].dt.year == year]
    # crosstab days by month
    df_explode = pd.crosstab(df_explode['Day'], [df_explode['Month'], df_explode['MonthName']])
    
    # get months
    col_index = df_explode.columns
       
    # get days in month of complete months in year
    calyear = calendar[calendar.date.dt.year == year].reset_index(drop=True)
    days_in_months = calyear.daysinmonths
    
    # months complete this year
    months = len(calyear)
    
    # initialize
    no_moviedays = []
    pct_no_moviedays = 0
    
    for i in range(months):
        moviedays = 0
        # sum all days per month with 0 - False (no movies watched)
        moviedays = df_explode[col_index[i]].astype(bool).sum(axis=0)
        # append sum to array (sum per month)
        no_moviedays.append(int(days_in_months[i] - moviedays))
    
    # Percentage of days per year without a movie watched
    if days_in_months.sum() != 0:
        pct_no_moviedays = int((sum(no_moviedays) / days_in_months.sum()) * 100)
    else:
        pct_no_moviedays = 0
        
    print(str(year))
    print('By month: ' + str(no_moviedays))
    print('Total: ' + str(sum(no_moviedays)) + '/' + str(calyear.daysinmonths.sum()))
    print(str(pct_no_moviedays) + ' %')
    print('\n')

In [29]:
# Create crosstab filtering by year
def crosstab_by_year(df, index=None, year=None):
    # if year=None get all years
    cols = ['Title','Date']
    
    if year != None:
        df_ = df[df['Date'].dt.year == year][cols].copy()
    else:
        df_ = df[cols].copy()
    
    df_ = explode_date(df_)
    if index == None:
        # Movies by month/year
        multiIndex = df_['DateYear']
    else:
        multiIndex = [df_[index], df_[index+'Name']]
    
    if index == 'Weekday' or index == None:
        ct = pd.crosstab(multiIndex, [df_['Month'], df_['MonthName']], margins=True)
    elif index == 'Month':
        ct = pd.crosstab(multiIndex, df_['Day'])
    
    return ct

In [30]:
# Movie ratio by movies watched on weekends vs movies watched on weekdays
def ratio_weekend(df, year=None):    
    df_ = create_weekday_df(df, year)

    # Create Weekdays df with weekdays' names
    allweek = df_.Weekday.values #['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']    
    df_week = pd.DataFrame(0, index=allweek, columns=['Movies'])    
    df_week.reset_index(inplace=True)
    df_week.rename(columns={"index": "Weekday"}, inplace=True)

    # Concatenate dfs
    df_concat = pd.concat([df_week, df_]).groupby(['Weekday']).sum().reset_index()
    df_concat = df_concat.set_index('Weekday').loc[allweek]

    # Weekdays vs Weekend
    dropdays = allweek[1:-1] # ['Tuesday','Wednesday','Thursday','Friday','Saturday']
    # weekdays sum
    df_concat.loc['Monday'] += df_concat.iloc[1:5].sum()
    # weekend sum
    df_concat.loc['Sunday'] += df_concat.iloc[5].sum()
    # drop unecessary columns
    df_concat.drop(dropdays, inplace=True)
    # rename columns
    df_concat.rename(index={'Monday': 'Weekdays'}, inplace=True)
    df_concat.rename(index={'Sunday': 'Weekend'}, inplace=True)
    
    # Get ratio
    ratio = df_concat.reset_index()['Movies']
    pct_ratio = ((ratio.iloc[1] / (ratio.iloc[0] + ratio.iloc[1])) * 100).astype(int)    
    print(str(pct_ratio) + '% of the movies were watched on the weekend!')

In [31]:
# Count occurrences of string
def counter_display(df, by, col, TOP=None):
    # Check type
    sample = df[by].sample().values[0]
    
    if isinstance(sample, str):
        series = df[by].str.split(', ').apply(lambda x: [e.strip() for e in x])
        # Remove duplicates
        series = series.map(lambda x: list(set(x)))
        series = pd.Series(series.map(Counter).sum())

    else: # list
        series = pd.Series(df[by].value_counts())
           
    # Organize df to export
    df_export = pd.DataFrame(series, columns=[col])
    
    if TOP == None:
        # Everything
        df_export = df_export.sort_values(by=col, ascending=False)
    else:
        # Show TOP
        df_export = df_export.nlargest(TOP, col, keep='all')
   
    return df_export

In [32]:
# Average movie rating by column
# If column values have multiple substrings in a string (examples: 'Genre', 'Actors', etc) use explode() to separate all values
def get_mean_value(df, by, get_avg, TOP=None):   
    # Create df to explode by 'by'
    df_explode = df.copy() # remove some not needed columns
    # Split multiple persons in one string
    df_explode[by] = df_explode[by].str.split(', ').apply(lambda x: [e.strip() for e in x])

    # Remove duplicates
    df_explode[by] = df_explode[by].map(lambda x: list(set(x)))
    
    # Explode df base on 'by'
    df_explode = explode(df_explode, [by])
    
    df_export = df_explode.groupby(by).agg({'Title':['count','; '.join], get_avg:'mean'})
    df_export = df_export.droplevel(0, axis=1)
    df_export = df_export.rename(columns={'join':'Movies', 'count':'Total', 'mean':get_avg})
    
    # if 'imdbRating'
    if get_avg.find('Rating') != -1:
        df_export[get_avg] = round(df_export[get_avg], 2)
    # else 'Runtime' or 'imdbVotes'
    else:
        df_export[get_avg] = df_export[get_avg].astype(int)
         
    if TOP == None:
        # Everything
        df_export = df_export.sort_values('Total', ascending=False)
    else:
        # Show TOP
        df_export = df_export.nlargest(TOP, 'Total', keep='all')
    
    return df_export

In [33]:
# Movies by actor
def count_actor(series, TOP):     
    seriesNew = []
    
    for i in range(len(series)):
        # initialize an empty string
        series.iloc[i]
        str1 = ', ' .join(series.iloc[i])
        seriesNew.append(Counter([x.strip() for x in str1.split(',')]).most_common(TOP))
                      
    return pd.Series(seriesNew)

In [34]:
# Pair director - actor
def director_actors(data, col, TOP):
    colList = counter_display(data, col, 'Actors', None).reset_index()['index']
    d = {}

    for i in range(len(colList)):
        value = colList[i]        
        d[value] = [data[data[col].str.contains(colList[i])]['Actors'].values]

    data_export = pd.DataFrame.from_dict(d, orient='index', columns=['Actors']).head(TOP)
    
    return data_export

In [35]:
# Check movies per day
def movies_per_day(df, year):    
    nMovies = df.loc[year]['Title']

    now = pd.Timestamp('now')
    if year == now.year:
        lastDay = date.today()
    else:
        lastDay = date(year, 12, 31)
    
    firstDay = date(year, 1, 1)
    delta = lastDay - firstDay

    moviesPerDay = nMovies / (delta.days + 1) # 1st of January
    print(moviesPerDay.round(2), 'movies per day in', year)

In [36]:
# Function that computes the weighted rating of each movie
def weighted_rating(v, R, m, C):
    # v - votes
    # R - Rating
    # m - minimun number of votes
    # C - mean
    score = []
    # Calculation based on the IMDB formula
    score = (v/(v+m) * R) + (m/(m+v) * C).round(3)
        
    return score

In [37]:
# Genre columns to use in the functions below
cols_genre = ['Genre','Title','imdbRating','Date']

# Genre with most watched movies
def most_watched_genre(df, TOP, year):
    df_explode_genre = df[df['Date'].dt.year == year][cols_genre].copy()
       
    df_explode_genre = split_explode(df_explode_genre, 'Genre')     
    df_explode_genrerat = get_mean_value(df_explode_genre, 'Genre', 'imdbRating')
    
    show_all(df_explode_genrerat.nlargest(TOP, 'Total', keep='all'))
    
    return df_explode_genrerat


# Most watched genre combo
def most_watched_genre_combo(df_, TOP, year):
    df_genre = df_[df_['Date'].dt.year == year][cols_genre].copy()
    
    gb_genre = df_genre.groupby(by='Genre')
    # Genre combination with most movies, showing worst and best rated movies
    df_genre_agg = gb_genre.agg({'imdbRating': ['min','max','mean'], 
                                 'Title': 'count'})
    df_genre_agg.reset_index(inplace=True)
    
    df_genre_agg_worst = gb_genre['imdbRating'].idxmin().fillna(0).astype(int)
    df_genre_agg[('Title','worst')] = df_genre.reindex(df_genre_agg_worst).reset_index(drop=True)['Title']
    
    df_genre_agg_best = gb_genre['imdbRating'].idxmax().fillna(0).astype(int)
    df_genre_agg[('Title','best')] = df_genre.reindex(df_genre_agg_best).reset_index(drop=True)['Title']
    
    df_genre_agg = df_genre_agg.set_index([('Genre','')]).round(1)
    df_genre_agg.index.rename('Genre', inplace=True)
    
    show_all(df_genre_agg.nlargest(TOP, [('Title','count')], keep='all'))
    
    return df_genre_agg

In [38]:
# Double group showing count, list of movies and average rating
def double_group_info(df, col1, col2, TOP=3):   
    # Explode columns
    cols = [col1, col2]
    
    for col in cols:
        df = split_explode(df, col)
    
    # Remove top 2 countries from dataframe
    list_remove_countries = ['United States','United Kingdom']
    if col1 == 'Country':
        df = df[~df[col1].isin(list_remove_countries)]
    else:
        pass
        
    # Clean strings
    df[col1] = df[col1].apply(lambda x : x.lstrip(' ').rstrip(' '))
    df[col2] = df[col2].apply(lambda x : x.lstrip(' ').rstrip(' '))
    
    # Drop duplicates, for instance, production companies with multiple appearences in same movie
    df = df[~df.duplicated()]
    
    # Group by
    gb_df = df.groupby([col1,col2]).agg({'Title':['count','; '.join], 'imdbRating':'mean'})
    gb_df[('imdbRating','mean')] = gb_df[('imdbRating','mean')].round(2)
    
    # Select only cases with more than 1 movie
    gb_df = gb_df[gb_df[('Title','count')] > 1]
    # Sort values by group
    gb_df = gb_df.groupby(level=0, group_keys=False).apply(lambda x: x.sort_values(('Title','count'), ascending=False).nlargest(TOP, ('Title','count'), keep='all')) #.head(TOP))
    
    #show_all(gb_df)
    
    return gb_df

In [39]:
# Common column values pairing
def pairing(df, col, pair=2):
    list_ = df[col].apply(lambda x: x.split(','))
    list_ = [[x.lstrip().rstrip() for x in l] for l in list_] # strip ' ' from strings
    
    d  = Counter()
    for sub in list_:
        if len(list_) < pair:
            continue
        sub.sort()
        for comb in combinations(sub, pair):
            d[comb] += 1
            
    df_pairs = pd.DataFrame.from_dict(d, orient='index').reset_index()
    df_pairs.columns = ['Pairs','Count']
    df_pairs = df_pairs.set_index('Pairs')
    
    return df_pairs

In [40]:
# Common column values pairing
def pairing_columns(df, col1, col2, pair=2):
    list_ = df[col1].apply(lambda x: x.split(','))
    list_ = [[x.lstrip().rstrip() for x in l] for l in list_] # strip ' ' from strings
    
    d  = Counter()
    for sub in list_:
        if len(list_) < pair:
            continue
        sub.sort()
        for comb in combinations(sub, pair):
            d[comb] += 1
            
    df_pairs = pd.DataFrame.from_dict(d, orient='index').reset_index()
    df_pairs.columns = ['Pairs','Count']
    
    return df_pairs

## Plotting

In [41]:
# Set colormap
plot_cmap = 'YlGnBu'

In [42]:
# Date related lists
days = np.arange(1, 32, 1) # 1 to 31
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [43]:
# Display everything
def show_all(df):
    display(HTML(df.to_html()))

In [44]:
# function to add value labels
def add_labels(x,y):
    for i in range(len(x)):
        plt.text(i, y[i], y[i], ha = 'center')

In [45]:
# Vertical bar plot with x as 'year' 
def add_ticks_vbarplot(x, y, ax):
    bars = ax.bar(x, y, width=0.8)

    for bar in bars:
        height = bar.get_height()
        label_x_pos = bar.get_x() + bar.get_width() / 2
        ax.text(label_x_pos, height+0.1, s=f'{height}', ha='center', va='bottom')
        
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(9)
    
    start = min(x)
    end = max(x)+1
    
    #start, end = ax.get_xlim()
    ax.xaxis.set_ticks(np.arange(start, end))
    ax.tick_params(axis='x', labelsize=10)
    ax.set_xticklabels(np.arange(start, end), rotation=90)   
    ax.yaxis.get_major_locator().set_params(integer=True)
    
    return ax

In [46]:
# Draw a heatmap with the numeric values in each cell
def plot_heatmap(df, index, year):
    DAYS = 31
    MONTHS = 12

    plotme = crosstab_by_year(df, index, year)
    plotme = plotme.droplevel(0, axis=0)

    # If less than 12 months, append missing months
    if index=='Weekday':
        # Drop rows used to organize data and remove 'All'
        plotme = plotme.iloc[:-1,:]
        plotme = plotme.droplevel(0, axis=1).iloc[:,:-1]
        
        if len(plotme.columns) < MONTHS:           
            # Create df to complement data
            df_12months = pd.DataFrame(0, index=plotme.index.values, columns=months[len(plotme.columns):])
            plotme = plotme.join(df_12months)
            
        # Set plot
        ylabel = index
        _, ax = plt.subplots(figsize=(9,4))
        plt.title('Movie ' + ylabel + ' count by Month')
        
    elif index=='Month':
        if len(plotme.index) < MONTHS:
            # Create df to complement data
            df_12months = pd.DataFrame(0, index=months[len(plotme.index):], columns=plotme.columns.values)
            plotme = plotme.append(df_12months)
            
        # Use days instead of weekdays
        plotme = plotme.T

        # Add missing days
        add_rows = list(set(np.arange(1, DAYS+1, 1)) - set(plotme.index))        
        plotme = plotme.append(pd.DataFrame(0, index=add_rows, columns=plotme.columns))
        plotme.sort_index(inplace=True)
        
        # Set plot
        ylabel = 'Day'
        _, ax = plt.subplots(figsize=(8,15))
        plt.title('Movie ' + ylabel + ' count by ' + index)

    # Plot
    sns.heatmap(plotme, annot=True, fmt="d")
    plt.xlabel('Month')
    plt.ylabel(ylabel)
    
    plt.show()

In [47]:
# Create week dataframe with total movies watched by weekday
def create_weekday_df(df, year):
    # Select 'All' column and remove 'All' row
    df_weekday = pd.DataFrame(crosstab_by_year(df, 'Weekday', year)[('All','')]).reset_index().iloc[:-1, :] 
    # Get columns names ('Weekday' and 'WeekdayName')
    df_weekday.columns = df_weekday.columns.droplevel(1)    
    # Select and rename columns
    df_weekday = df_weekday[['WeekdayName','All']]
    df_weekday.rename(columns={"WeekdayName": "Weekday", "All": "Movies"}, inplace=True)    

    return df_weekday

In [48]:
# Movies by Weekday
def plot_weekday_bar(df, year):    
    df_weekday = create_weekday_df(df, year)

    # Prepare plot
    cmap = plt.get_cmap(plot_cmap)
    norm = plt.Normalize(df_weekday['Movies'].min(), df_weekday['Movies'].max())
    values = df_weekday['Movies'].values
    
    # Plotting
    ax = plt.figure(figsize=(len(values),6)).gca()
    sns.barplot('Weekday', 'Movies', data=df_weekday, palette=cmap(norm(values)))
    
    ax.yaxis.get_major_locator().set_params(integer=True)
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 90) 

    plt.title('# Movies watched by Weekday')
    plt.ylabel('# Movies')
    
    for i, n in enumerate(df_weekday['Movies']):
        plt.text(i, n+0.3, n, fontdict={'fontsize':12})
    
    plt.show()
    
    # Plot ratio
    ratio_weekend(df, year)

In [49]:
# Movies by Week of Year
def plot_week_bar(df, year):  
    NWEEKS = 52
    nweeks = np.arange(1, NWEEKS+1, 1)
    
    df_woy = df[['Title','Date']].copy()
    df_woy['Weekofyear'] = df_woy['Date'].dt.weekofyear
    
    # No movies from week 53 in the beginning of the year
    temp = df_woy[df_woy['Date'].dt.year == year]
    temp = temp[~((temp['Date'].dt.month == 1) & (temp['Date'].dt.weekofyear >= NWEEKS))]
    
    # Week starts on monday and some values may fall on week 52 or 53 of previous year
    if df_woy[df_woy['Date'].dt.year == year+1].size > 0:
        appendthis = df_woy[df_woy['Date'].dt.year == year+1][df_woy['Date'].dt.weekofyear >= NWEEKS]
        temp = temp.append(appendthis)
        
    # Group by
    df_woy = temp.copy()
    gb_woy = df_woy.groupby('Weekofyear').agg({'Title':'count'}).reset_index()
    
    # If not all weeks of year present, add them
    add_rows = list(set(nweeks) - set(gb_woy.Weekofyear))
    gb_woy = gb_woy.append(pd.DataFrame({'Weekofyear': add_rows, 'Title': 0}))
    gb_woy.sort_values('Weekofyear', inplace=True)

    # Prepare plot
    cmap = plt.get_cmap(plot_cmap)
    norm = plt.Normalize(gb_woy['Title'].min(), gb_woy['Title'].max())
    values = gb_woy['Title'].values
    
    gb_woy.Weekofyear = gb_woy.Weekofyear.astype(int)
    gb_woy.rename(columns={'Title':'# Movies'}, inplace=True)
    
    # Plotting     
    plotme = gb_woy.set_index('Weekofyear').T
    xsize = NWEEKS // 2
    plt.figure(figsize=(xsize, 2))
    sns.heatmap(plotme, annot=True, fmt="d", cbar_kws={"orientation": "horizontal", "pad": 0.5})
    
    plt.title('# Movies watched by Week of Year (in ' + str(year) +')')
    plt.xlabel('Week of Year')
    #plt.ylabel('# Movies') 
    
    plt.show()

In [50]:
# Plot how many hours spent watching movies by month
def plot_hour_month_bar(df, year):
    # Group by month (get .month to maintain month order)
    group_month = [df['Date'].dt.month, df['Date'].dt.month_name()]
    
    df_month = df[df['Date'].dt.year==year].groupby(group_month).agg({'Runtime':'sum'}).droplevel(0, axis=0).reset_index()
    df_month['Runtime'] = (df_month['Runtime'] / 60).astype(int)
    df_month = df_month.rename(columns={'Date':'Month'})

    # If less than 12 months, append missing months
    if len(df_month) < len(months):
        df_12months = pd.DataFrame({'Month':months, 'Runtime':0})
        df_month = df_month.append(df_12months.iloc[len(df_month):,:])
    
    # Prepare plot
    cmap = plt.get_cmap(plot_cmap)
    norm = plt.Normalize(df_month.Runtime.min(), df_month.Runtime.max())
    values = df_month.Runtime.values

    # Plotting
    ax = plt.figure(figsize=(len(months),6)).gca()
    sns.barplot(x='Month', y='Runtime',data=df_month, palette=cmap(norm(values)))
    
    ax.yaxis.get_major_locator().set_params(integer=True)
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 90) 
    
    plt.title('# Hours of movies watched by Month (in ' + str(year) +')')
    plt.ylabel('# Hours')
    
    for i, n in enumerate(values):
        plt.text(i, n+0.1, n, fontdict={'fontsize':12})
        
    plt.show()

In [51]:
# Plot swarm box plot with x based on Release Year or Decade
def plot_box_swarm(x, y, data, order):
    # Plot
    xsize = len(order)
    
    _, ax = plt.subplots(figsize=(xsize, 6))
    ax = sns.boxplot(x, y, data=data, order=order)
    ax = sns.swarmplot(x, y, data=data, order=order, color=".4")
    
    # add grid lines
    ax.yaxis.grid(True)
    ax.xaxis.grid(True)
   
    if x == 'Year':
        x = 'Release Year'
        ax.set_xlabel(x)
        
    title = y + ' by ' + x    
    ax.set_title(title)    
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 90) 

In [52]:
# Plot number of Movies by Release Year
def plot_movie_trend(df, year=None):
    if year is not None:
        plot_movies_year = df[df['Date'].dt.year == year].groupby(by='Year').agg({'Title': 'count'}).reset_index()
    else:
        plot_movies_year = df.copy()
        
    x = plot_movies_year['Year']
    y = plot_movies_year['Title']
    
    # Plot
    # https://towardsdatascience.com/how-to-make-bar-and-hbar-charts-with-labels-using-matplotlib-b701ce70ba9c   
    _, ax = plt.subplots(figsize=(30, 6))
    add_ticks_vbarplot(x, y, ax)     
    
    if year is not None:
        plt.title('# Movies watched by Release Year (in ' + str(year) +')')
    else:
        plt.title('# Movies watched by Release Year')
        
    plt.xlabel('Release Year')
    plt.ylabel('# Movies')
    
    plt.show()